In [1]:

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
address = 'Rio de Janeiro, Rio de Janeiro'

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rio de Janeiro is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rio de Janeiro is -22.9110137, -43.2093727.


In [3]:
# Set up Foursquare
CLIENT_ID = 'XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK' # your Foursquare ID
CLIENT_SECRET = 'HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK
CLIENT_SECRET:HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV


In [4]:

search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [5]:

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK&client_secret=HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV&ll=-22.9110137,-43.2093727&v=20180605&query=Restaurant&radius=50000&limit=100'

In [6]:
results = requests.get(url).json()


In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",False,56e04e1d38fa3ae11155b863,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,RJ,Riá Restaurant,v-1588188396
1,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4ece6782cc219860f521dd5d,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,RJ,Restaurante Pitada,v-1588188396
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,50e0b634e4b0f94bd8a38f47,Navio Soberano,BR,Rio de Janeiro,Brasil,NaN,2849,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.8928508209919...",-22.892851,-43.189795,NaN,RJ,Restaurant El Duero,v-1588188396
3,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4d46d2732e326ea8e41afba6,"R. do Matoso, 30",BR,Rio de Janeiro,Brasil,Pç. da Bandeira,435,"[R. do Matoso, 30 (Pç. da Bandeira), Rio de Ja...","[{'label': 'display', 'lat': -22.9120948222039...",-22.912095,-43.213454,NaN,RJ,Restaurante Grão & Grão,v-1588188396
4,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4e0209e4b0fbdf39a4592821,Silva paulet esquina com costa barros,BR,Fortaleza,Brasil,Aldeota,431,[Silva paulet esquina com costa barros (Aldeot...,"[{'label': 'display', 'lat': -22.9148909, 'lng...",-22.914891,-43.209306,NaN,Ce,Restaurante Braga,v-1588188396


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Riá Restaurant,Bistro,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,RJ,56e04e1d38fa3ae11155b863
1,Restaurante Pitada,Brazilian Restaurant,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,RJ,4ece6782cc219860f521dd5d
2,Restaurant El Duero,Restaurant,Navio Soberano,BR,Rio de Janeiro,Brasil,NaN,2849,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.8928508209919...",-22.892851,-43.189795,NaN,RJ,50e0b634e4b0f94bd8a38f47
3,Restaurante Grão & Grão,Brazilian Restaurant,"R. do Matoso, 30",BR,Rio de Janeiro,Brasil,Pç. da Bandeira,435,"[R. do Matoso, 30 (Pç. da Bandeira), Rio de Ja...","[{'label': 'display', 'lat': -22.9120948222039...",-22.912095,-43.213454,NaN,RJ,4d46d2732e326ea8e41afba6
4,Restaurante Braga,Restaurant,Silva paulet esquina com costa barros,BR,Fortaleza,Brasil,Aldeota,431,[Silva paulet esquina com costa barros (Aldeot...,"[{'label': 'display', 'lat': -22.9148909, 'lng...",-22.914891,-43.209306,NaN,Ce,4e0209e4b0fbdf39a4592821


In [17]:
! conda install -c conda-forge folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTAL

In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map